# Build your image and define your own component with KFP

In [1]:
import kfp
import kfp.gcp as gcp
import kfp.dsl as dsl
import kfp.compiler as compiler
import kfp.components as comp
import datetime

import kubernetes as k8s

In [2]:
client = kfp.Client()

## Build Reusable Components
### Writing the program code

two imput arguments
--model_file: 
--bucket: 

作為路徑名稱
`model.save(model_file)` 、
`gcs_path = bucket + "/" + model_file`

gcs 儲存訓練相關日誌，供後續的 tensorboard 取用

In [4]:
%%bash

# Create folders if they don't exist.
mkdir -p tmp/reuse_components/mnist_training

# Create the Python file that lists GCS blobs.
cat > ./tmp/reuse_components/mnist_training/app.py <<HERE
import argparse
from datetime import datetime
import tensorflow as tf

parser = argparse.ArgumentParser()
parser.add_argument(
    '--model_file', type=str, required=True, help='Name of the model file.')
parser.add_argument(
    '--bucket', type=str, required=True, help='GCS bucket name.')
args = parser.parse_args()

bucket=args.bucket
model_file=args.model_file

model = tf.keras.models.Sequential([
  tf.keras.layers.Flatten(input_shape=(28, 28)),
  tf.keras.layers.Dense(512, activation=tf.nn.relu),
  tf.keras.layers.Dropout(0.2),
  tf.keras.layers.Dense(10, activation=tf.nn.softmax)
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

print(model.summary())    

mnist = tf.keras.datasets.mnist
(x_train, y_train),(x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0

callbacks = [
  tf.keras.callbacks.TensorBoard(log_dir=bucket + '/logs/' + datetime.now().date().__str__()),
  # Interrupt training if val_loss stops improving for over 2 epochs
  tf.keras.callbacks.EarlyStopping(patience=2, monitor='val_loss'),
]

model.fit(x_train, y_train, batch_size=32, epochs=5, callbacks=callbacks,
          validation_data=(x_test, y_test))


model.save(model_file)

from tensorflow import gfile

gcs_path = bucket + "/" + model_file

if gfile.Exists(gcs_path):
    gfile.Remove(gcs_path)

gfile.Copy(model_file, gcs_path)
with open('/output.txt', 'w') as f:
  f.write(gcs_path)
HERE

## Build Docker image with KFP and notebook in the K8s

### Creating a Dockerfile

In [7]:
%%bash

# Create Dockerfile.
cat > ./tmp/reuse_components/mnist_training/Dockerfile <<EOF
FROM tensorflow/tensorflow:1.15.0-py3
WORKDIR /app
COPY . /app
EOF

#### Use Kaniko to build image in the K8s
We are going to use the kfp.containers.build_image_from_working_dir to build the image and push to the Container Registry (GCR), which uses **kaniko**.

```
IMAGE_NAME="mnist_training_kf_pipeline"
TAG="latest" # "v_$(date +%Y%m%d_%H%M%S)"
PROJECT_ID="infuseai-dev"
GCS_BUCKET="gs://jack-kubeflow-gcs"

GCR_IMAGE="gcr.io/{PROJECT_ID}/{IMAGE_NAME}:{TAG}".format(
    PROJECT_ID=PROJECT_ID,
    IMAGE_NAME=IMAGE_NAME,
    TAG=TAG
)

builder = kfp.containers._container_builder.ContainerBuilder(
    gcs_staging=GCS_BUCKET + "/kfp_container_build_staging")

image_name = kfp.containers.build_image_from_working_dir(
    image_name=GCR_IMAGE,
    working_dir='./tmp/reuse_components/mnist_training/',
    builder=builder
)

image_name
```

#### Use docker to build image in local env

```shell=
%%bash -s "{PROJECT_ID}"

IMAGE_NAME="mnist_training_kf_pipeline"
TAG="latest" # "v_$(date +%Y%m%d_%H%M%S)"

# Create script to build docker image and push it.
cat > ./tmp/components/mnist_training/build_image.sh <<HERE
PROJECT_ID="${1}"
IMAGE_NAME="${IMAGE_NAME}"
TAG="${TAG}"
GCR_IMAGE="gcr.io/\${PROJECT_ID}/\${IMAGE_NAME}:\${TAG}"
docker build -t \${IMAGE_NAME} .
docker tag \${IMAGE_NAME} \${GCR_IMAGE}
docker push \${GCR_IMAGE}
docker image rm \${IMAGE_NAME}
docker image rm \${GCR_IMAGE}
HERE

cd tmp/components/mnist_training
bash build_image.sh
```

### Writing your component definition file

Create Yaml
the image uri should be changed according to the above docker image push output

`mnist_component.yaml`

```yaml
name: Mnist training
description: Train a mnist model and save to GCS
inputs:
  - name: model_file
    description: 'Name of the model file.'
    type: String
  - name: bucket
    description: 'GCS bucket name.'
    type: String
outputs:
  - name: model_path
    description: 'Trained model path.'
    type: GCSPath
implementation:
  container:
    image: ${GCR_IMAGE}
    command: [
      python, /app/app.py,
      --model_file, {inputValue: model_file},
      --bucket,     {inputValue: bucket},
    ]
    fileOutputs:
      model_path: /output.txt
```

### Load your component

In [ ]:
import os
mnist_train_op = kfp.components.load_component_from_file(os.path.join('./', 'mnist_pipeline_component.yaml'))


### Load third party defined component

In [ ]:
mlengine_deploy_op = comp.load_component_from_url(
    'https://raw.githubusercontent.com/kubeflow/pipelines/3f4b80127f35e40760eeb1813ce1d3f641502222/components/gcp/ml_engine/deploy/component.yaml')

# define a function to accept input argument and return the component
def deploy(
    project_id,
    model_uri,
    model_id,
    runtime_version,
    python_version):
    
    return mlengine_deploy_op(
        model_uri=model_uri,
        project_id=project_id, 
        model_id=model_id, 
        runtime_version=runtime_version, 
        python_version=python_version,
        replace_existing_version=True, 
        set_default=True)

### Write a function to test prediction (func to component)

In [30]:
def deployment_test(project_id: str, model_name: str, version: str) -> str:

    model_name = model_name.split("/")[-1]
    version = version.split("/")[-1]
    
    import googleapiclient.discovery
    
    def predict(project, model, data, version=None):
      """Run predictions on a list of instances.

      Args:
        project: (str), project where the Cloud ML Engine Model is deployed.
        model: (str), model name.
        data: ([[any]]), list of input instances, where each input instance is a
          list of attributes.
        version: str, version of the model to target.

      Returns:
        Mapping[str: any]: dictionary of prediction results defined by the model.
      """

      service = googleapiclient.discovery.build('ml', 'v1')
      name = 'projects/{}/models/{}'.format(project, model)

      if version is not None:
        name += '/versions/{}'.format(version)

      response = service.projects().predict(
          name=name, body={
              'instances': data
          }).execute()

      if 'error' in response:
        raise RuntimeError(response['error'])

      return response['predictions']

    import tensorflow as tf
    import json
    
    mnist = tf.keras.datasets.mnist
    (x_train, y_train),(x_test, y_test) = mnist.load_data()
    x_train, x_test = x_train / 255.0, x_test / 255.0

    result = predict(
        project=project_id,
        model=model_name,
        data=x_test[0:2].tolist(),
        version=version)
    print(result)
    
    return json.dumps(result)

In [31]:
deployment_test_op = comp.func_to_container_op(
    func=deployment_test, 
    base_image="tensorflow/tensorflow:1.15.0-py3",
    packages_to_install=["google-api-python-client==1.7.8"]) # define package to install

## Define the pipeline

What is apply
https://kubeflow-pipelines.readthedocs.io/en/latest/source/kfp.extensions.html#kfp.onprem.mount_pvc

In [ ]:
# Define the pipeline
@dsl.pipeline(
   name='Mnist pipeline',
   description='A toy pipeline that performs mnist model training.'
)
def mnist_reuse_component_deploy_pipeline(
    project_id: str = PROJECT_ID,
    model_path: str = 'mnist_model', 
    bucket: str = GCS_BUCKET
):
    train_task = mnist_train_op(
        model_path=model_path, 
        bucket=bucket
    ).apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    deploy_task = deploy(
        project_id=project_id,
        model_uri=train_task.outputs['gcs_model_path'],
        model_id="mnist", 
        runtime_version="1.14",
        python_version="3.5"
    ).apply(gcp.use_gcp_secret('user-gcp-sa'))  
    
    deploy_test_task = deployment_test_op(
        project_id=project_id,
        model_name=deploy_task.outputs["model_name"], 
        version=deploy_task.outputs["version_name"],
    ).apply(gcp.use_gcp_secret('user-gcp-sa'))
    
    return True

## Submit a pipeline run

In [ ]:
pipeline_func = mnist_reuse_component_deploy_pipeline
experiment_name = 'minist_kubeflow'

arguments = {"model_path":"mnist_model",
             "bucket":GCS_BUCKET}

run_name = pipeline_func.__name__ + ' run'

# Submit pipeline directly from pipeline function
run_result = client.create_run_from_pipeline_func(pipeline_func, 
                                                  experiment_name=experiment_name, 
                                                  run_name=run_name, 
                                                  arguments=arguments)